In [90]:
import numpy as np
import math
from scipy.integrate import quad
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interact, FloatSlider
from scipy.stats import norm

 $$C(K,T) = e^{-rT}\int_{K}^{\infty} (S_T - K)f_{\mathbb{Q}}(S_T)dS_T$$

**Heston (Stochastic Volatility)**

 
 $$
  dS_t = rS_t dt + \sqrt{v_t}S_t dW^1_t \\
  dv_t = \kappa(\theta - v_t)dt + \xi\sqrt{v_t}dW^2_t \\
  dW^1_t dW^2_t = \rho dt
 $$

##### Fast Fourier Transform
 
 $$X_k = \sum_{n=0}^{N-1} x_n e^{\frac{-i2 \pi k n}{N}}$$
 
 The Fast Fourier Transform (FFT) is an efficient algorithm for computing the DFT that reduces complexity from $O(N^2)$ to $O(N \log N)$ by:

 - Recursively dividing the transform into smaller sub-transforms
 - Exploiting symmetries in the complex exponentials
 - Using a divide-and-conquer approach

 Key properties:
 - Input size must be a power of 2 ($N = 2^m$)
 - Widely used in signal processing, numerical computing, and financial applications
 - Most common variant is the Cooley-Tukey FFT algorithm

##### Inverse Fast Fourier Transform
  
  $$x_n = \frac{1}{N} \sum_{k=0}^{N-1} X_k e^{\frac{i2 \pi k n}{N}}$$
  
  The Inverse Fast Fourier Transform (IFFT) reverses the FFT operation, transforming from frequency domain back to time domain:

  - Uses same efficient $O(N \log N)$ algorithm as FFT
  - Only difference is positive exponent and $\frac{1}{N}$ scaling factor
  - Preserves all properties of FFT including power of 2 input size requirement

  The IFFT allows us to recover the original signal after performing frequency domain operations

##### 🃏 Characteristic Functions


#### Fourier Transform: Characteristic Function of a Random Variable

 The characteristic function $\phi_X(t) = \mathbb{E}[e^{itX}]$ transforms a probability distribution into the frequency domain:
 
  $$\phi_X(t) = \mathbb{E}[e^{itX}] = \int_{-\infty}^{\infty} e^{itx} f_X(x) dx$$

There is a sign difference, however these mathematical objects (characteristic function, Fourier transform) are roughly interchangable

 Key properties of characteristic functions:

 - Uniquely determines the probability distribution of a random variable
 - Always exists for any random variable (unlike moment generating functions)
 - Provides one-to-one correspondence with PDF via Fourier transform
 - Enables derivation of all distribution moments
 - Useful for distributions lacking closed-form density functions
 - Simplifies calculations through properties like sum of independent variables

 The characteristic function serves as a bridge between probability theory and frequency analysis

##### 🎯 Carr-Madan

Revisiting the option pricing problem now in the Heston framework, we have the following pricing problem

 
 $$C(K,T) = e^{-rT}\mathbb{E}^{\mathbb{Q}}\left[\max(S_T - K, 0)\right]$$
 
 where the expectation is defined as
 
$$\mathbb{E}^{\mathbb{Q}}\left[\max(S_T - K, 0)\right] = \int_{-\infty}^{\infty} \max(S_T - K, 0) f_{S_T}(x) dx = \int_{K}^{\infty} (S_T - K) f_{S_T}(x) dx$$
 
What if we were to apply a Fourier transform to this call option value?

Well we run into an issue - the standard call option pricing function is NOT *square-integrable* which is necessary for the Fourier transform 

This is due to the log-strike approaching a constant at $-\infty$ which is not zero

#### Damping Factor $e^{\alpha k}$

By introducing a damping factor, Carr-Madan enable square-integrability of the pricing function and thus the Fourier transform applies

 Let $k = \ln(K)$ and $s = \ln(S_0)$, then the damped call price is:

 $$c_T(k) = e^{\alpha k}C_T(k)$$

 where $\alpha > 0$ is the damping parameter. The modified call price $c_T(k)$ is now square-integrable.

 The Fourier transform of the damped call price is:

 $$\psi_T(v) = \int_{-\infty}^{\infty} e^{ivk}c_T(k)dk$$

 And the inverse transform gives us the original call price:

 $$C_T(k) = \frac{e^{-\alpha k}}{2\pi}\int_{-\infty}^{\infty} e^{-ivk}\psi_T(v)dv$$

#### Key Equation

 The key equation for the Fourier transform of the damped call price in terms of the characteristic function is:

  Starting with the Fourier transform of the damped call price:
  $$\psi_T(v) = \int_{-\infty}^{\infty} e^{ivk}c_T(k)dk = \int_{-\infty}^{\infty} e^{ivk}e^{\alpha k}C_T(k)dk$$

  Substituting the call price formula:
  $$\psi_T(v) = \int_{-\infty}^{\infty} e^{ivk}e^{\alpha k}e^{-rT}\int_k^{\infty} (e^y - e^k)f_{S_T}(y)dy\,dk$$

  Changing order of integration:
  $$\psi_T(v) = e^{-rT}\int_{-\infty}^{\infty}\int_{-\infty}^y e^{ivk}e^{\alpha k}(e^y - e^k)dk\,f_{S_T}(y)dy$$

  Evaluating the inner integral:
  $$\psi_T(v) = e^{-rT}\int_{-\infty}^{\infty}\left[\frac{e^y e^{(\alpha+iv)k}}{(\alpha+iv)} - \frac{e^{(\alpha+iv+1)k}}{(\alpha+iv+1)}\right]_{-\infty}^y f_{S_T}(y)dy$$

  Simplifying:
  $$\psi_T(v) = e^{-rT}\int_{-\infty}^{\infty} e^y\left(\frac{e^{(\alpha+iv)y}}{(\alpha+iv)} - \frac{e^{(\alpha+iv+1)y}}{(\alpha+iv+1)}\right)f_{S_T}(y)dy$$

  Recognizing the characteristic function:
  $$\psi_T(v) = \frac{e^{-rT}}{(\alpha+iv)(\alpha+iv+1)}\phi_T(v-(i\alpha+1))S_0^{iv+\alpha+1}$$

 where $\phi_T(u)$ is the characteristic function of the log-price process.

Fortunately, the characteristic equation is known for the Heston model

 The characteristic function for the Heston model (log-stock prices) is:

 $$\phi_T(u) = \exp\left(iu(s + rT) + \frac{v_0}{\sigma^2}\left(\frac{1-e^{-dT}}{1-ge^{-dT}}\right)(a-\rho\sigma iu-d)\right) \times \exp\left(\frac{\theta\kappa}{\sigma^2}\left(2\ln\left(\frac{1-ge^{-dT}}{1-g}\right) + (a-\rho\sigma iu-d)T\right)\right)$$

 where:
 - $d = \sqrt{(\rho\sigma iu - a)^2 + \sigma^2(iu + u^2)}$
 - $g = \frac{a-\rho\sigma iu-d}{a-\rho\sigma iu+d}$
 - $a = \kappa - \rho\sigma iu$

 Parameters:
 - $\kappa$: mean reversion rate
 - $\theta$: long-run variance
 - $\sigma$: volatility of variance
 - $\rho$: correlation between asset and variance processes
 - $v_0$: initial variance
 - $s$: log spot price
 - $r$: risk-free rate
 - $T$: time to maturity

 To obtain the option price, we need to invert the Fourier transform:

 $$C_T(k) = \frac{e^{-\alpha k}}{2\pi}\int_{-\infty}^{\infty} e^{-ivk}\psi_T(v)dv$$

 Substituting the expression for $\psi_T(v)$:

 $$C_T(k) = \frac{e^{-\alpha k}}{2\pi}\int_{-\infty}^{\infty} e^{-ivk}\frac{e^{-rT}}{(\alpha+iv)(\alpha+iv+1)}\phi_T(v-(i\alpha+1))S_0^{iv+\alpha+1}dv$$

 This integral can be evaluated numerically using the Fast Fourier Transform (FFT).

 The key steps are:
 1. Choose a suitable damping parameter $\alpha > 0$ to ensure integrability
 2. Discretize the integral using a grid of points
 3. Apply FFT to compute the integral efficiently
 4. Undamp the result by multiplying by $e^{-\alpha k}$

 The resulting $C_T(k)$ gives us the call option price for log-strike $k$. 
 For actual strikes $K$, we use the relationship $k = \ln(K)$.


#### Putting the Pieces Together

- Carr-Madan introduce a damping factor to enable the Fourier transform of option prices

- They identify a relationship between the characteristic function of a pricing framework and the Fourier transform of the damped price

- Applying the IFT and undamping and correcting yields the price of options for then a *set* of strikes

In [92]:
@dataclass
class HestonParams:
    kappa: float
    sigma: float
    theta: float
    v0: float
    rho: float

# Characteristic function in log-strike
def heston_cf(u, T, S0, r, q, p: HestonParams):
    """
    Characteristic function φ(u) = E[exp(i u ln S_T)] under Q.
    Uses the Little Heston Trap parametrization.
    u can be scalar or numpy array.
    """
    i = 1j
    x0 = np.log(S0)
    a  = p.kappa * p.theta
    b  = p.kappa - p.rho * p.sigma * i * u
    d  = np.sqrt(b*b + (p.sigma**2) * (i*u + u*u))
    g  = (b - d) / (b + d)

    eDT = np.exp(-d * T)
    one_minus_g_eDT = 1 - g * eDT
    one_minus_g     = 1 - g
    # small guards
    one_minus_g_eDT = np.where(np.abs(one_minus_g_eDT) < 1e-15, 1e-15, one_minus_g_eDT)
    one_minus_g     = np.where(np.abs(one_minus_g)     < 1e-15, 1e-15, one_minus_g)

    C = i*u*(r - q)*T + (a/(p.sigma**2)) * ((b - d)*T - 2.0*np.log(one_minus_g_eDT/one_minus_g))
    D = ((b - d)/(p.sigma**2)) * ((1 - eDT) / one_minus_g_eDT)
    return np.exp(C + D*p.v0 + i*u*x0)
def simpson_weights(N: int):
    if N % 2 != 0:
        raise ValueError("N must be even")
    w = np.ones(N)
    w[1:N-1:2] = 4
    w[2:N-2:2] = 2
    return w
# alpha damping
# eta frequency step
def heston_fft_calls(S0: float, T: float, r:float, q:float, p:HestonParams, N:int = 4096, eta:float = 0.25, alpha:float = 1.5):
    # Will return (N,) ascending strikes
    # and (N,) call prices for those strikes
    n = np.arange(N)
    v = eta * n # frequency grid
    i = 1j
    phi_shift = heston_cf(v - (alpha + 1)*i, T, S0, r, q, p)
    denom = (alpha**2 + alpha - v**2 + i*(2*alpha + 1)*v)
    psi = np.exp(-r*T) * phi_shift / denom

    # Simpson weights for the v-integral
    w = simpson_weights(N) * (eta / 3.0)

    # FFT coupling
    lam = 2.0 * np.pi / (N * eta)   # Δk (log-strike step)
    b   = 0.5 * N * lam             # half-width in k
    x   = psi * np.exp(1j * b * v) * w

    F   = np.fft.fft(x)
    F   = np.real(F)

    j = np.arange(N)
    k = -b + j * lam                 # k = ln K
    K = np.exp(k)

    calls = np.exp(-alpha * k) / np.pi * F
    order = np.argsort(K)
    return K[order], np.maximum(calls[order], 0.0)

def heston_fft_call_price(
    S0: float, K: float, T: float, r: float, q: float, p: HestonParams,
    N: int = 4096, eta: float = 0.25, alpha: float = 1.5
):
    """Price a single call via FFT + linear interpolation on the K-grid."""
    K_grid, C_grid = heston_fft_calls(S0, T, r, q, p, N=N, eta=eta, alpha=alpha)
    if K <= K_grid[0]:
        return C_grid[0]
    if K >= K_grid[-1]:
        return C_grid[-1]
    idx = np.searchsorted(K_grid, K)
    x0, x1 = K_grid[idx-1], K_grid[idx]
    y0, y1 = C_grid[idx-1], C_grid[idx]
    return y0 + (y1 - y0) * (K - x0) / (x1 - x0)

def heston_fft_put_price(
    S0: float, K: float, T: float, r: float, q: float, p: HestonParams,
    N: int = 4096, eta: float = 0.25, alpha: float = 1.5
):
    """Put via put–call parity."""
    C = heston_fft_call_price(S0, K, T, r, q, p, N=N, eta=eta, alpha=alpha)
    return C - S0*np.exp(-q*T) + K*np.exp(-r*T)

In [96]:
def European_Call(S0, K, T, r , sigma):
    d1 = (np.log(S0/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return  S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_vega(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.pdf(d1) * np.sqrt(T)

def get_implied_vol(market_price, S0, K, T, r, max_iter = 200, tolerance = 1e-05):
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = European_Call(S0, K, T, r, sigma)
        diff = bs_price - market_price
        if abs(diff) < tolerance:
            return sigma
        vega = bs_vega(S0,K,T,r,sigma)
        if vega == 0:
            return None
        sigma -= diff/vega
    return None

In [ ]:

def plot_BS_implied_vol(S0, T, r, q, kappa, sigma, theta, v0, rho, K_min = 60, K_max = 300):
    fig, ax = plt.subplots()
    
    hp = HestonParams(kappa = kappa, sigma = sigma, theta = theta, v0 = v0, rho = rho)
    moneyness_grid = np.log(np.linspace(K_min , K_max, 300)/(S0 * np.exp(r * T)))
    K_grid = np.linspace(K_min, K_max, 300)
   
    
    heston_prices = [heston_fft_call_price(S0, k, T, r , q, hp) for k in K_grid]
    BS_implied_vols = [get_implied_vol(heston_price, S0, strike, T, r) * 100 for strike, heston_price in zip(K_grid,heston_prices)] 
    ax.scatter(moneyness_grid, BS_implied_vols, marker = "x", color = 'black')
    ax.set_xlabel("Moneyness")
    ax.set_ylabel("BS implied volatility (%)")
    ax.set_title("BS implied volatility(%) for Heston model")
    ax.grid(True)
    plt.show()

#plot_BS_implied_vol(S0 = 100, T = 1.0, r = 0.02, q = 0.0, kappa = 2.0, theta = 0.04,  sigma = 0.3, rho = -0.7, v0 = 0.05208)

interact(plot_BS_implied_vol,
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         kappa=FloatSlider(value=1.5, min=0.1, max=10.0, step=0.1, description="kappa"),
         sigma=FloatSlider(value=0.12, min=0.001, max=0.5, step=0.005, description="sigma"),
         theta=FloatSlider(value=0.01, min=0.01, max=2.0, step=0.01, description="theta"),
         v0=FloatSlider(value=0.1, min=0, max=0.3, step=0.01, description="v0"),
         rho=FloatSlider(value=0.00, min=-1.00, max=1.00, step=0.005, description="rho"))
       

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_vol(S0, T, r, q, kappa, sigma, theta, v0, rho, K_min=60, K_max=300)>